## Krylov.jl: A Julia basket of hand-picked Krylov methods

_STFC-Rutherford Appleton Laboratory, England_

_Thursday December 8, 2022_

**_Alexis Montoison_** (alexis.montoison@polymtl.ca), **_Dominique Orban_** (dominique.orban@polymtl.ca)

### How a Julia code can be both generic and efficient?
### How to support variable precision and architectures?

In [ ]:
using Pkg, LinearAlgebra, SparseArrays
pkg"activate ."   # Activate the environment with the file Project.toml at `.`
pkg"instantiate"  # Download all the packages declared in the Manifest.toml.

In [ ]:
pkg"status"  # Print out the status of the project/manifest.

In [ ]:
additional_packages = false
if additional_packages
  Pkg.add("BFloat16s")    # https://github.com/JuliaMath/BFloat16s.jl
  Pkg.add("MultiFloats")  # https://github.com/dzhang314/MultiFloats.jl   
end

In [ ]:
gpu = false
if gpu
  Pkg.add("CUDA")    # Nvidia GPUs
  Pkg.add("AMDGPU")  # AMD GPUs
  Pkg.add("OneAPI")  # Intel GPUs
  Pkg.add("Metal")   # Apple M1 GPUs
end

In [ ]:
pkg"up" # Update all packages of the environment

## I) Introduction

[Krylov.jl](https://github.com/JuliaSmoothOptimizers/Krylov.jl) is a Julia package that implements a collection of Krylov processes and methods for solving a variety of linear problems:

|  Square systems | Linear least-squares problems |Linear least-norm problems              |
|:---------------:|:-----------------------------:|:--------------------------------------:|
| $Ax = b$        | $\min \|b - Ax\|$             |$\min \|x\|  \text{  s.t.  }  Ax = b$ |

| Adjoint systems | Saddle-point and Hermitian quasi-definite systems | Generalized saddle-point and non-Hermitian partitioned systems |
|:---------------:|:-------------------------------------------------:|:--------------------------------------------------------------:|
|$\begin{matrix} Ax = b \\ A^{H} y = c \end{matrix}$ | $\begin{bmatrix} M & A \\ A^{H} & -N \end{bmatrix} \begin{bmatrix} x \\ y \end{bmatrix} = \begin{bmatrix} b \\ c \end{bmatrix}$ | $\begin{bmatrix} M & A \\ B & N \end{bmatrix} \begin{bmatrix} x \\ y \end{bmatrix} = \begin{bmatrix} b \\ c \end{bmatrix}$ |

$A^{H\!}$ denotes the conjugate transpose of $A$.
It coincides with $A^{T\!}$, the transpose of $A$, if $A$ is real.
Krylov methods are iterative methods based on Krylov subspaces.

They are an alternative to direct methods such as Gaussian elimination or QR decomposition when storage requirements or computational costs become prohibitive, which is often the case for large and sparse linear problems.

Contrary to direct methods, which require storing $A$ explicitly, Krylov methods support linear operators to model operator-vector products $u \leftarrow Av$, and in some instances $u \leftarrow A^{H\!}w$ because Krylov processes only require those operations to build Krylov subspaces.

The same goes with preconditioners, i.e., transformations that modify a linear system into an 
equivalent form with favorable spectral properties that may yield faster convergence in finite-precision arithmetic.

__Cayley-Hamilton theorem__:
If $A$ is a square matrix of size $n$ and $p(X) = \det(XI_n - A) = X^n + p_{n-1} X^{n-1} + \dots + p_1 X + p_0$ is its characteristic polynomial, then $p(A) = A^n + p_{n-1} A^{n-1} + \dots + p_1 A + p_0 I_n = 0_n$.
<br/><br/>
If $A$ is nonsingular, $p_0 \ne 0$ and
$$A^{-1} = -\dfrac{1}{p_0}(A^{n-1} + p_{n-1} A^{n-2} + \dots + p_1 I_n)$$
<br/><br/>
$$x = A^{-1}b \implies x\in K_n(A, b) = \mathop{\mathrm{Span}} \{b, Ab, \dots, A^{n-1}b\}$$
<br/><br/>
$K_n(A, b)$ is a *Krylov subspace*.

__Principle of Krylov methods__: Build iteratively a solution $x_k \in K_k(A,b)$ of $Ax=b$.
<br/><br/>
A process is used to build an (orthogonal) basis of $K_k(A, b)$.
We have the Lanczos process for Hermitian matrices and the Arnoldi process for square non-Hermitian matrices.

The projection of $A$ into the Krylov subspace has a workable structure and $x_k = V_k y_k$, where $y_k \in \mathbb{R}^k$ is the solution of a subproblem that uses the projection of $A$.

When $A$ is rectangular, we use the Golub-Kahan process to build orthogonal bases of $K_k(A^T A, A^T b)$ and $K_k(A A^T, b)$ and we use the normal equations to solve linear least-squares and least-norm problems.

We refer interested readers to [ipsen-meyer-1998](https://doi.org/10.1080/00029890.1998.12004985) for an introduction to Krylov methods along with [greenbaum-1997](https://doi.org/10.1137/1.9781611970937) and [saad-2003](https://doi.org/10.1137/1.9780898718003) for more details.

### SuiteSparse Matrix Collection

The [SuiteSparse Matrix Collection](https://sparse.tamu.edu/) (previously UFL collection) gathers about 3000 problems from multiple fields and of multiple sizes. The collection is often used in scientific papers. It allows easy testing and comparison of new implementations of direct and iterative methods.
<br/><br/>
Each problem is stored in the format *MatrixMarket* (.mtx), *MAT* (.mat) and *Rutherford-Boeing* (.rb).
An interface to the SuiteSparse Matrix Collection exists in Julia.

In [ ]:
using SuiteSparseMatrixCollection
using HarwellRutherfordBoeing
using MatrixMarket
using MAT

In [ ]:
# Database
ssmc = ssmc_db(verbose=false)

In [ ]:
spd = ssmc[(ssmc.numerical_symmetry .== 1) .& (ssmc.positive_definite.== true) .& (ssmc.real .== true).& (ssmc.nrows .≤ 100), :]
paths = fetch_ssmc(spd, format="MM")

In [ ]:
path = paths[1]
A = MatrixMarket.mmread(joinpath(path, "$(spd.name[1]).mtx"))

In [ ]:
function get_matrix(name :: String)
    # Get information about the matrix
    pb = ssmc_matrices(ssmc, "", name)
    
    # Download the matrix
    paths = fetch_ssmc(pb, format="MM")
    path_mtx = paths[1]

    # Load the matrix
    mtx = MatrixMarket.mmread(joinpath(path_mtx, "$name.mtx"))
    return mtx
end

In [ ]:
M = get_matrix("sherman5")

##  II) Largest collection of Krylov processes and methods

Krylov.jl aims to provide a unified interface for the largest collection of Krylov processes and methods, all programming languages taken together, with six and thirty-three implementations, respectively:

- **Krylov processes**: [_Arnoldi_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/processes/#Arnoldi), [_Golub-Kahan_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/processes/#Golub-Kahan), [_Hermitian Lanczos_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/processes/#Hermitian-Lanczos), [_Montoison-Orban_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/processes/#Montoison-Orban), [_Non-Hermitian Lanczos_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/processes/#Non-Hermitian-Lanczos), [_Saunders-Simon-Yip_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/processes/#Saunders-Simon-Yip);
- **Krylov methods**: [_Bicgstab_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/unsymmetric/#BiCGSTAB), [_Bilq_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/unsymmetric/#BiLQ), [_Bilqr_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/as/#BiLQR), [_Cg_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/spd/#CG), [_Cg-lanczos_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/spd/#CG-LANCZOS), [_Cg-lanczos-shift_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/spd/#CG-LANCZOS-SHIFT), [_Cgls_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/ls/#CGLS), [_Cgne_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/ln/#CGNE), [_Cgs_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/unsymmetric/#CGS), [_Cr_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/spd/#CR), [_Craig_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/ln/#CRAIG), [_Craigmr_](), [_Crls_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/ls/#CRLS), [_Crmr_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/ln/#CRMR), [_Diom_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/unsymmetric/#DIOM), [_Dqgmres_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/unsymmetric/#DQGMRES), [_Fgmres_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/unsymmetric/#FGMRES), [_Fom_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/unsymmetric/#FOM), [_Gmres_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/unsymmetric/#GMRES), [_Gpmr_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/gsp/#GPMR), [_Lnlq_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/ln/#LNLQ), [_Lslq_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/ls/#LSLQ), [_Lsmr_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/ls/#LSMR), [_Lsqr_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/ls/#LSQR), [_Minres_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/sid/#MINRES), [_Minres-qlp_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/sid/#MINRES-QLP), [_Qmr_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/unsymmetric/#QMR), [_Symmlq_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/sid/#SYMMLQ), [_Tricg_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/sp_sqd/#TriCG), [_Trilqr_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/as/#TriLQR), [_Trimr_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/sp_sqd/#TriMR), [_Usymlq_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/ln/#CRAIGMR), [_Usymqr_](https://juliasmoothoptimizers.github.io/Krylov.jl/dev/solvers/ls/#USYMQR).

Some processes and methods are not available elsewhere and are the product of our own research. References for each process and method are available in the extensive [documentation](https://juliasmoothoptimizers.github.io/Krylov.jl/stable/).

In [ ]:
using Krylov
using Plots

- Hermitian positive definite linear systems (**cg**, **cr**, **cg_lanczos**)

In [ ]:
A = get_matrix("1138_bus")
n, m = size(A)
b = ones(n)
x, stats = cg(A, b, history=true)
x

In [ ]:
plot(0:stats.niter, stats.residuals, label="CG", xlabel="k", ylabel="‖rₖ‖", yaxis=:log10)

In [ ]:
norm(b - A*x)

- Hermitian indefinite linear systems (**symmlq**, **minres**, **minres_qlp**)

In [ ]:
A = get_matrix("meg4")
n, m = size(A)
b = ones(n)
x, stats = cg(A, b, history=true)  # symmlq, minres
stats

In [ ]:
plot(0:stats.niter, stats.residuals, label="CG", xlabel="k", ylabel="‖rₖ‖", yaxis=:log10)

In [ ]:
norm(b - A*x)

- Square non-Hermitian linear systems (**bilq**, **qmr**, **usymlq**, **usymqr**, **cgs**, **bicgstab**, **diom**, **fom**, **dqgmres**, **gmres**, **fgmres**)

In [ ]:
A = get_matrix("orani678")
n, m = size(A)
b = ones(n)
x, stats = gmres(A, b, history=true)
stats

In [ ]:
plot(0:stats.niter, stats.residuals, label="GMRES", xlabel="k", ylabel="‖rₖ‖", yaxis=:log10)

In [ ]:
norm(b - A*x)

- Least-norm problems (**cgne**, **crmr**, **lnlq**, **craig**, **craigmr**, **usymlq**)

In [ ]:
A = get_matrix("well1033")'
n, m = size(A)
b = rand(n)
x, y, stats = craigmr(A, b, history=true)
stats

In [ ]:
norm(b - A*A'*y)

In [ ]:
norm(b - A*x)

- Least-squares problems (**cgls**, **crls**, **lslq**, **lsqr**, **lsmr**, **usymqr**)

In [ ]:
A = get_matrix("well1850")
n, m = size(A)
b = rand(n)
x, stats = lsqr(A, b, history=true)
stats

In [ ]:
norm(A'*b - A'A*x)

- Adjoint systems (**bilqr**, **trilqr**)

In [ ]:
A = get_matrix("arc130")
n, m = size(A)
b = rand(n)
c = rand(m)
x, y, stats = bilqr(A, b, c)

In [ ]:
norm(b - A*x)

In [ ]:
norm(c - A'*y)

In [ ]:
A = get_matrix("illc1033")
m, n = size(A)
b = ones(m)
c = -ones(n)

K = [I A; A' -I]
d = [b; c]

(x, y, stats) = tricg(A, b, c)
r =  d - K * [x; y]
norm(r)

In [ ]:
M = diagm(0 => [3.0 * i for i = 1:m])
N = diagm(0 => [5.0 * i for i = 1:n])
M⁻¹ = inv(M)
N⁻¹ = inv(N)

K = [M A; A' -N]
(x, y, stats) = tricg(A, b, c, M=M⁻¹, N=N⁻¹)
r =  d - K * [x; y]
norm(r)

- Generalized saddle-point and non-Hermitian partitioned systems (**gpmr**)

In [ ]:
A = [1.0 0.0; 0.0 -1.0; 3.0 0.0]
B = [0.0 2.0 4.0; -3.0 0.0 0.0]
n, m = size(A)
b = ones(n)
c = -ones(m)

K = [I A; B I]
d = [b; c]

x, y, stats = gpmr(A, B, b, c)
r =  d - K * [x; y]
norm(r)

In [ ]:
M = diagm(0 => [2.0 * i for i = 1:n])
N = diagm(0 => [16.0 * i for i = 1:m])
M⁻¹ = inv(M)
N⁻¹ = inv(N)

K = [M A; B N]
x, y, stats = gpmr(A, B, b, c, C=M⁻¹, D=N⁻¹)
r =  d - K * [x; y]
norm(r)

## III) Support for any floating-point system supported by Julia

Krylov.jl works with real and complex data in any floating-point system supported by Julia, which means that Krylov.jl handles any precision `T` and `Complex{T}` where `T <: AbstractFloat`.

Although most personal computers offer IEEE 754 single and double precision computations, new architectures implement native computations in other floating-point systems.

In addition, software libraries such as the GNU MPFR, shipped with Julia, let users experiment with computations in variable, extended precision at the software level with the `BigFloat` data type.

Working in high precision has obvious benefits in terms of accuracy.

In [ ]:
using Quadmath, DoubleFloats

In [ ]:
A = get_matrix("685_bus")
n, m = size(A)
b = rand(n)

In [ ]:
precision = Float32
A2 = precision.(A)
b2 = precision.(b)
x, stats = cg(A2, b2, history=true, atol=zero(precision), rtol=zero(precision), itmax=1000)
p = plot(0:stats.niter, stats.residuals, label=string(precision), xlabel="k", ylabel="‖rₖ‖", yaxis=:log10)

In [ ]:
precision = Float64
A2 = precision.(A)
b2 = precision.(b)
x, stats = cg(A2, b2, history=true, atol=zero(precision), rtol=zero(precision), itmax=1000)
p = plot(0:stats.niter, stats.residuals, label=string(precision), xlabel="k", ylabel="‖rₖ‖", yaxis=:log10)

In [ ]:
precision = Double64
A2 = precision.(A)
b2 = precision.(b)
x, stats = cg(A2, b2, history=true, atol=zero(precision), rtol=zero(precision), itmax=1000)
p = plot(0:stats.niter, stats.residuals, label=string(precision), xlabel="k", ylabel="‖rₖ‖", yaxis=:log10)

In [ ]:
precision = Float128
A2 = precision.(A)
b2 = precision.(b)
x, stats = cg(A2, b2, history=true, atol=zero(precision), rtol=zero(precision), itmax=1000)
plot(0:stats.niter, stats.residuals, label=string(precision), xlabel="k", ylabel="‖rₖ‖", yaxis=:log10)

In [ ]:
precision = BigFloat
A2 = precision.(A)
b2 = precision.(b)
x, stats = cg(A2, b2, history=true, atol=zero(precision), rtol=zero(precision), itmax=1000)
plot(0:stats.niter, stats.residuals, label=string(precision), xlabel="k", ylabel="‖rₖ‖", yaxis=:log10)

## IV) Support for Nvidia, AMD and Intel GPUs

Krylov methods are well suited for GPU computations because they only require operator-vector products ($u \leftarrow Av$, $u \leftarrow A^{H\!}w$) and vector operations ($\|v\|$, $u^H v$, $v \leftarrow \alpha u + \beta v$), which are highly parallelizable.

The implementations in Krylov.jl are generic so as to take advantage of the multiple dispatch and broadcast features of Julia.

Those allow the implementations to be specialized automatically by the compiler for both CPU and GPU.

Thus, Krylov.jl works with GPU backends that build on [GPUArrays.jl](https://github.com/JuliaGPU/GPUArrays.jl), such as [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl), [AMDGPU.jl](https://github.com/JuliaGPU/AMDGPU.jl), [oneAPI.jl](https://github.com/JuliaGPU/oneAPI.jl) or [Metal.jl](https://github.com/JuliaGPU/Metal.jl).

![layers](https://user-images.githubusercontent.com/35051714/114203656-5b082700-9926-11eb-9332-1282c3f32348.png)

```julia
V=Vector    # CPU vector
V=CuVector  # GPU CUDA vector
V=ROCVector # GPU ROCm vector
V=oneArray  # GPU oneAPI vector

function incrmul(a::AbstractArray,
                 b::AbstractArray)
                 c::AbstractArray)
  c .+= a .* b
end
incrmul(a, b, c)
```

All solvers in Krylov.jl can be used with [CUDA.jl](https://github.com/JuliaGPU/CUDA.jl) and allow computations on Nvidia GPUs.
Problems stored in CPU format (`Matrix` and `Vector`) must first be converted to the related GPU format (`CuMatrix` and `CuVector`).

In [ ]:
using CUDA, Krylov

# CPU Arrays
A_cpu = rand(20, 20)
b_cpu = rand(20)

# GPU Arrays
A_gpu = CuMatrix(A_cpu)
b_gpu = CuVector(b_cpu)

# Solve a square and dense system on an Nivida GPU
x, stats = bilq(A_gpu, b_gpu)

Sparse matrices have a specific storage on Nvidia GPUs (`CuSparseMatrixCSC`, `CuSparseMatrixCSR` or `CuSparseMatrixCOO`):


In [ ]:
using CUDA, Krylov
using CUDA.CUSPARSE, SparseArrays

# CPU Arrays
A_cpu = sprand(200, 100, 0.3)
b_cpu = rand(200)

# GPU Arrays
A_gpu = CuSparseMatrixCSC(A_cpu)
b_gpu = CuVector(b_cpu)

# Solve a rectangular and sparse system on an Nvidia GPU
x, stats = lsmr(A_gpu, b_gpu)

Sparse matrices have a specific storage on Nvidia GPUs (`CuSparseMatrixCSC`, `CuSparseMatrixCSR` or `CuSparseMatrixCOO`):

In [ ]:
using Krylov, AMDGPU

# CPU Arrays
A_cpu = rand(ComplexF64, 20, 20)
A_cpu = A_cpu + A_cpu'
b_cpu = rand(ComplexF64, 20)

A_gpu = ROCMatrix(A_cpu)
b_gpu = ROCVector(b_cpu)

# Solve a dense Hermitian system on an AMD GPU
x, stats = minres(A_gpu, b_gpu)

All solvers in Krylov.jl, can be used with [oneAPI.jl](https://github.com/JuliaGPU/oneAPI.jl) and allow computations on Intel GPUs.
Problems stored in CPU format (`Matrix` and `Vector`) must first be converted to the related GPU format (`oneMatrix` and `oneVector`).

In [ ]:
using Krylov, oneAPI

T = Float32  # oneAPI.jl also works with ComplexF32
m = 20
n = 10

# CPU Arrays
A_cpu = rand(T, m, n)
b_cpu = rand(T, m)

# GPU Arrays
A_gpu = oneMatrix(A_cpu)
b_gpu = oneVector(b_cpu)

# Solve a dense least-squares problem on an Intel GPU
x, stats = lsqr(A_gpu, b_gpu)

All solvers in Krylov.jl, can be used with [Metal.jl](https://github.com/JuliaGPU/Metal.jl) and allow computations on Apple M1 GPUs.
Problems stored in CPU format (`Matrix` and `Vector`) must first be converted to the related GPU format (`MtlMatrix` and `MtlVector`).

In [ ]:
using Krylov, Metal

T = Float32  # Metal.jl also works with ComplexF32
n = 10
m = 20

# CPU Arrays
A_cpu = rand(T, n, m)
b_cpu = rand(T, n)

# GPU Arrays
A_gpu = MtlMatrix(A_cpu)
b_gpu = MtlVector(b_cpu)

# Solve a dense least-norm problem on an Apple M1 GPU
x, stats = craig(A_gpu, b_gpu)

## V) Support for linear operators

The input arguments of all Krylov.jl solvers that model $A$, $B$, $M$, $N$ and preconditioners can be any object that represents a linear operator.

Krylov methods combined with linear operators allow to reduce computation time and memory requirements considerably by avoiding building and storing matrices.

In nonlinear optimization, finding a critical point of a continuous function frequently involves linear systems where $A$ is a Hessian or a Jacobian.

Materializing such operators as matrices is expensive in terms of operations and memory consumption and is unreasonable for high-dimensional problems.

However, it is often possible to implement efficient Hessian-vector and Jacobian-vector products, for example with the help of automatic differentiation tools.

## VI) In-place methods

All solvers in Krylov.jl have an in-place variant that allows to solve multiple linear systems with the same dimensions, precision and architecture.

Optimization methods such as the Newton and Gauss-Newton methods can take advantage of this functionality by allocating workspace for the solve only once.

The in-place variants only require a Julia structure that contains all the storage needed by a Krylov method as additional argument.

In-place methods limit memory allocations and deallocations, which are particularly expensive on GPUs.

All solvers in Krylov.jl have an in-place variant implemented in a method whose name ends with `!`.
A workspace (`KrylovSolver`) that contains the storage needed by a Krylov method can be used to solve multiple linear systems that have the same dimensions in the same floating-point precision.
Each `KrylovSolver` has two constructors:

```julia
XyzSolver(A, b)
XyzSolver(m, n, S)

```

`Xyz` is the name of the Krylov method with lowercase letters except its first one (`Cg`, `Minres`, `Lsmr`, `Bicgstab`, ...).
Given an operator `A` and a right-hand side `b`, you can create a `KrylovSolver` based on the size of `A` and the type of `b` or explicitly give the dimensions `(m, n)` and the storage type `S`.

For example, use `S = Vector{Float64}` if you want to solve linear systems in double precision on the CPU.

The workspace is always the first argument of the in-place methods:

```julia
minres_solver = MinresSolver(n, n, Vector{Float64})
minres!(minres_solver, A, b)
```

## VII) Performance optimizations

Operator-vector products and vector operations are the most expensive operations in Krylov.jl.

We rely on BLAS routines as much as possible to perform those operations.

By default, Julia ships with OpenBLAS and provides multithreaded routines.

Since Julia 1.6, users can also switch dynamically to other BLAS backends, such as the Intel MKL or BLIS, thanks to the BLAS demuxing library `libblastrampoline`, if an optimized BLAS is available.

In [ ]:
using LinearAlgebra
BLAS.get_config()

In [ ]:
import LinearAlgebra, OpenBLAS32_jll
LinearAlgebra.BLAS.lbt_forward(OpenBLAS32_jll.libopenblas_path)
BLAS.get_config()

In [ ]:
import LinearAlgebra, MKL_jll
LinearAlgebra.BLAS.lbt_forward(MKL_jll.libmkl_rt_path)
BLAS.get_config()

In [ ]:
import LinearAlgebra, blis_jll
LinearAlgebra.BLAS.lbt_forward(blis_jll.blis_path)
BLAS.get_config()

In [ ]:
osx = false
if osx
  blas = "/System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/libBLAS.dylib"
  lapack = "/System/Library/Frameworks/Accelerate.framework/Versions/A/Frameworks/vecLib.framework/libLAPACK.dylib"
  LinearAlgebra.BLAS.lbt_forward(blas, clear=true, verbose=true)
  LinearAlgebra.BLAS.lbt_forward(lapack, clear=false, verbose=true)
end
BLAS.get_config()

## VIII) Storage requirements

A “Storage Requirements” section is available in the documentation to provide the theoretical number of bytes required by each method.

Our implementations are storage-optimal in the sense that they are guaranteed to match the theoretical storage amount.

The match is verified in the unit tests by way of functions that return the number of bytes allocated by our implementations.

In [ ]:
using Krylov

m = 5000
n = 12000
A = rand(Float64, m, n)
b = rand(Float64, m)
solver = LsmrSolver(A, b)
show(stdout, solver, show_stats=false)

In [ ]:
nbytes = sizeof(solver)

In [ ]:
FC = Float64                            # precision of the least-squares problem
ncoefs_lsmr = 5*n + 2*m                 # number of coefficients
nbytes_lsmr = sizeof(FC) * ncoefs_lsmr  # number of bytes

## IX) Examples

Our first example is a simple implementation of the Gauss-Newton method without linesearch for nonlinear least squares.
It illustrates several of the facilities of Krylov.jl: solver preallocation and reuse, genericity with respect to data types, and linear operators.

<br/><br/>
$$\left.J(x_k) u = \dfrac{d}{dt} F(x_k + tu) \right|_{t=0}$$
<br/><br/>
$$\left.J^T(x_k) w = \dfrac{d}{dx} F(x)^T w \right|_{x=x_k}$$
<br/><br/>

In [ ]:
using LinearAlgebra    # Linear algebra library of Julia
using SparseArrays     # Sparse library of Julia
using Krylov           # Krylov methods and processes
using LinearOperators  # Linear operators
using ForwardDiff      # Automatic differentiation
using Quadmath         # Quadruple precision
using MKL              # Intel BLAS

"The Gauss-Newton method for Nonlinear Least Squares"
function gauss_newton(F, JF, x₀::AbstractVector{T}; itmax = 200, tol = √eps(T)) where T
    n = length(x₀)
    x = copy(x₀)
    Fx = F(x)
    m = length(Fx)
    iter = 0
    S = typeof(x)                 # precision and architecture
    solver = LsmrSolver(m, n, S)  # structure that contains the workspace of LSMR
    solved = tired = false
    while !(solved || tired)
        Jx = JF(x)              # Compute J(xₖ)
        lsmr!(solver, Jx, -Fx)  # Minimize ‖J(xₖ)Δx + F(xₖ)‖
        x .+= solver.x          # Update xₖ₊₁ = xₖ + Δx
        Fx_old = Fx             # F(xₖ)
        Fx = F(x)               # F(xₖ₊₁)
        iter += 1
        solved = norm(Fx - Fx_old) / norm(Fx) ≤ tol
        tired = iter ≥ itmax
    end
    return x
end

T = Float128  # IEEE quadruple precision
x₀ = ones(T, 2)
F(x) = [x[1]^4 - 3; exp(x[2]) - 2; log(x[1]) - x[2]^2]         # F(x)
J(y, x, v) = ForwardDiff.derivative!(y, t -> F(x + t * v), 0)  # y ← JF(x)v
Jᵀ(y, x, w) = ForwardDiff.gradient!(y, x -> dot(F(x), w), x)   # y ← JFᵀ(x)w
symmetric = hermitian = false
JF(x) = LinearOperator(T, 3, 2, symmetric, hermitian, (y, v) -> J(y, x, v),   # non-transpose
                                                      (y, w) -> Jᵀ(y, x, w),  # transpose
                                                      (y, w) -> Jᵀ(y, x, w))  # conjugate transpose
gauss_newton(F, JF, x₀)

Another example based on a simplistic Newton method without linesearch for convex optimization:

$$\left.\nabla^2 f(x_k) v = \dfrac{d}{dt} \nabla f(x_k + tv) \right|_{t=0}$$

In [ ]:
"The Newton method for convex optimization"
function newton(∇f, ∇²f, x₀::AbstractVector{T}; itmax = 200, tol = √eps(T)) where T
    n = length(x₀)
    x = copy(x₀)
    gx = ∇f(x)
    iter = 0
    S = typeof(x)               # precision and architecture
    solver = CgSolver(n, n, S)  # structure that contains the workspace of CG
    solved = tired = false
    while !(solved || tired)
        Hx = ∇²f(x)           # Compute ∇²f(xₖ)
        cg!(solver, Hx, -gx)  # Solve ∇²f(xₖ)Δx = -∇f(xₖ)
        x .+= solver.x        # Update xₖ₊₁ = xₖ + Δx
        gx = ∇f(x)            # ∇f(xₖ₊₁)
        iter += 1
        solved = norm(gx) ≤ tol
        tired = iter ≥ itmax
    end
    return x
end
T = Float16  # IEEE half precision
n = 4
x₀ = -ones(T, n)
f(x) = sum((x[i] - i)^2 for i = 1:n)                                # f(x)
∇f(x) = ForwardDiff.gradient(f, x)                                  # ∇f(x)
H(y, x, v) = ForwardDiff.derivative!(y, t -> ∇f(x + t * v), 0)      # y ← ∇²f(x)v
symmetric = hermitian = true
∇²f(x) = LinearOperator(T, n, n, symmetric, hermitian, (y, v) -> H(y, x, v))  # ∇²f(x)
newton(∇f, ∇²f, x₀)

Our second example concerns the solution of a complex Hermitian linear system from the [SuiteSparse Matrix Collection](https://sparse.tamu.edu/) with an incomplete Cholesky factorization preconditioner on GPU.

The preconditioner $P$ is implemented as an in-place linear operator that performs the forward and backward sweeps with the Cholesky factor to model $P^{-1}$.

Because the system matrix is Hermitian and positive definite, we use the conjugate gradient method.

However, other methods for Hermitian systems could be used, including _Symmlq_, _Cr_ and _Minres_.

In [ ]:
using LinearAlgebra                # Linear algebra library of Julia
using SparseArrays                 # Sparse library of Julia
using Krylov                       # Krylov methods and processes
using LinearOperators              # Linear operators
using MatrixMarket                 # Reader of matrices stored in the Matrix Market format
using SuiteSparseMatrixCollection  # Interface to the SuiteSparse Matrix Collection
using CUDA                         # Interface to Nvidia GPUs
using CUDA.CUSPARSE                # Nvidia CUSPARSE library

ssmc = ssmc_db()
matrices = ssmc_matrices(ssmc, "Bai", "mhd1280b")
paths = fetch_ssmc(matrices, format="MM")
path_A = joinpath(paths[1], "mhd1280b.mtx")
A_cpu = MatrixMarket.mmread(path_A)
m, n = size(A_cpu)
b_cpu = ones(ComplexF64, m)

# Transfer the linear system from the CPU to the GPU
A_gpu = CuSparseMatrixCSR(A_cpu)
b_gpu = CuVector(b_cpu)

# Incomplete Cholesky decomposition LLᴴ ≈ A with zero fill-in
P = ic02(A_gpu, 'O')

# Solve Py = x
function ldiv_ic0!(y, P, x)
  copyto!(y, x)
  ldiv!(LowerTriangular(P), y)   # Forward substitution with L
  ldiv!(LowerTriangular(P)', y)  # Backward substitution with Lᴴ
  return y
end

# Linear operator that model the preconditioner P⁻¹
T = ComplexF64
symmetric = false
hermitian = true
P⁻¹ = LinearOperator(T, m, n, symmetric, hermitian, (y, x) -> ldiv_ic0!(y, P, x))

# Solve a Hermitian positive definite system with an incomplete Cholesky factorization preconditioner
x, stats = cg(A_gpu, b_gpu, M=P⁻¹)